In [1]:
import pandas as pd
import shap
import xgboost
from sklearn.preprocessing import OneHotEncoder
from old_shap import DataHandler

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Read in the data
life_expectancy = DataHandler(csv_path="data/life_expectancy.csv")

In [3]:
life_expectancy.drop_columns(["Country Code"])

,Country Name,Region,IncomeGroup,Year,Life Expectancy World Bank,Prevelance of Undernourishment,CO2,Health Expenditure %,Education Expenditure %,Unemployment,Corruption,Sanitation,Injuries,Communicable,NonCommunicable
0,Afghanistan,South Asia,Low income,2001,56.308,47.8,730.0,NaN,NaN,10.809000,NaN,NaN,2179727.10,9689193.70,5795426.38
1,Angola,Sub-Saharan Africa,Lower middle income,2001,47.059,67.5,15960.0,4.483516,NaN,4.004000,NaN,NaN,1392080.71,11190210.53,2663516.34
2,Albania,Europe & Central Asia,Upper middle income,2001,74.288,4.9,3230.0,7.139524,3.4587,18.575001,NaN,40.520895,117081.67,140894.78,532324.75
3,Andorra,Europe & Central Asia,High income,2001,NaN,NaN,520.0,5.865939,NaN,NaN,NaN,21.788660,1697.99,695.56,13636.64
4,United Arab Emirates,Middle East & North Africa,High income,2001,74.544,2.8,97200.0,2.484370,NaN,2.493000,NaN,NaN,144678.14,65271.91,481740.70


In [4]:
life_expectancy.get_pct_missing()

,pct_missing
Corruption,0.705082
Sanitation,0.377193
Education Expenditure %,0.329704
Prevelance of Undernourishment,0.206897
Unemployment,0.091954
Life Expectancy World Bank,0.056866
Health Expenditure %,0.054446
CO2,0.045977
Country Name,0.000000
Region,0.000000


In [5]:
life_expectancy.one_hot_encode(features=["IncomeGroup"], concat=True)
life_expectancy.drop_columns(["IncomeGroup"])

2023-09-13 19:22:58.060 | INFO     | old_shap:one_hot_encode:24 - Number of new features: 4


,Country Name,Region,Year,Life Expectancy World Bank,Prevelance of Undernourishment,CO2,Health Expenditure %,Education Expenditure %,Unemployment,Corruption,Sanitation,Injuries,Communicable,NonCommunicable,IncomeGroup_High income,IncomeGroup_Low income,IncomeGroup_Lower middle income,IncomeGroup_Upper middle income
0,Afghanistan,South Asia,2001,56.308,47.8,730.0,NaN,NaN,10.809000,NaN,NaN,2179727.10,9689193.70,5795426.38,0.0,1.0,0.0,0.0
1,Angola,Sub-Saharan Africa,2001,47.059,67.5,15960.0,4.483516,NaN,4.004000,NaN,NaN,1392080.71,11190210.53,2663516.34,0.0,0.0,1.0,0.0
2,Albania,Europe & Central Asia,2001,74.288,4.9,3230.0,7.139524,3.4587,18.575001,NaN,40.520895,117081.67,140894.78,532324.75,0.0,0.0,0.0,1.0
3,Andorra,Europe & Central Asia,2001,NaN,NaN,520.0,5.865939,NaN,NaN,NaN,21.788660,1697.99,695.56,13636.64,1.0,0.0,0.0,0.0
4,United Arab Emirates,Middle East & North Africa,2001,74.544,2.8,97200.0,2.484370,NaN,2.493000,NaN,NaN,144678.14,65271.91,481740.70,1.0,0.0,0.0,0.0


In [ ]:
# Get a percentage of missing values
life_expectancy.isna().sum() / len(life_expectancy) * 100

In [ ]:
# Let's one-hot encode the IncomeGroup column
income_group_ohe = one_hot_encode(life_expectancy, ['IncomeGroup'])
life_expectancy = pd.concat([life_expectancy, income_group_ohe], axis=1)
life_expectancy.drop(columns=['IncomeGroup'], inplace=True)

In [ ]:
# We can drop the country column as it is not useful for our model
life_expectancy = life_expectancy.drop(columns=['Country', 'Country Code'])

In [ ]:
# Let's one hot encode the IncomeGroup column
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
income_group_ohe = ohe.fit_transform(life_expectancy[['IncomeGroup']])
income_group_ohe = pd.DataFrame(income_group_ohe, columns=ohe.get_feature_names_out(['IncomeGroup']))
life_expectancy = pd.concat([life_expectancy, income_group_ohe], axis=1)


In [ ]:
life_expectancy

In [ ]:
# train an XGBoost model
X, y = shap.datasets.california()
model = xgboost.XGBRegressor().fit(X, y)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:, "Latitude"], color=shap_values)

In [ ]:
shap.plots.beeswarm(shap_values)